# Worthäufigkeiten

Brauchen von Stopwords bereinigte lemmatisierte Texte

Schritte:
- Texte laden
- Wörter zählen
- als DF mit Wort|Freq ausgeben

In [1]:
import pandas as pd
meta = pd.read_csv("meta_epik.csv", encoding = "utf8")
meta.head()

,Unnamed: 0,id,title,author,period,type,genre,date,file,source,annotation,tokens_cleaned
0,0,1,Der sterbende Cato,Gottsched,Aufklaerung,Tragedy,drama,1731,gottsched-der-sterbende-cato.xml,https://github.com/dracor-org/gerdracor,NaN,22047
1,1,2,Ein Deutsches Vorspiel,Neuber,Aufklaerung,NaN,drama,1734,neuber-ein-deutsches-vorspiel.xml,https://github.com/dracor-org/gerdracor,NaN,6480
2,2,3,Die Pietisterey im Fischbein-Rocke oder Die Do...,Gottsched,Aufklaerung,Comedy,drama,1736,gottschedin-die-pietisterey-im-fischbein-rocke...,https://github.com/dracor-org/gerdracor,NaN,27691
3,3,4,Die von der Weisheit wider die Unwissenheit be...,Neuber,Aufklaerung,NaN,drama,1736,neuber-die-beschuetzte-schauspielkunst.xml,https://github.com/dracor-org/gerdracor,NaN,7377
4,4,5,Die Verehrung der Vollkommenheit durch die geb...,Neuber,Aufklaerung,NaN,drama,1737,neuber-die-verehrung-der-vollkommenheit.xml,https://github.com/dracor-org/gerdracor,NaN,7091


In [2]:
def load_corpus(path, df):
    from numpy import append 

    texts = []
    filenames = []
    for filename in df["file"]:
        with open(path + filename, 'r', encoding="utf16") as f:
            texts.append(f.read())
            f.close()
        filenames.append(filename)    
    return texts, filenames

In [3]:
def load_corpus_lyrik(path):
    from numpy import append 
    from os import listdir
    
    texts = []
    filenames = []
    for filename in listdir(path):
        with open(path + "/" + filename, 'r', encoding="utf16") as f:
            texts.append(f.read())

        filenames.append(filename)    
    return texts, filenames

In [4]:
# texts_aufklaerung, filenames_aufklaerung = load_corpus("corpora/cleaned_normalized/corpus_drama/", meta.loc[meta["period"]=="Aufklaerung"])
# texts_kunstepoche, filenames_kunstepoche = load_corpus("corpora/cleaned_normalized/corpus_drama/", meta.loc[meta["period"]=="Kunstepoche"])
# texts, filenames = load_corpus("corpora/cleaned_normalized/corpus_drama/", meta)

In [5]:
texts, filenames = load_corpus_lyrik("corpora/cleaned_normalized/corpus_lyrik/")

In [6]:
texts_aufklaerung = []
texts_kunstepoche = []

for i in range(len(texts)):
    if "_aufklaerung" in filenames[i]:
        texts_aufklaerung.append(texts[i])
    else:
        texts_kunstepoche.append(texts[i])

In [7]:
len(texts_aufklaerung)

72

In [8]:
len(texts_kunstepoche)

76

In [9]:
texts[0][:100]

'Wodan Donnerer sinken Eroberer Tyrann schlank deutsch Todesspeer -- ha -- hieben löwenmutig Todeswun'

In [10]:
len(texts_aufklaerung)

72

## make first counter of all texts

In [11]:
df = pd.DataFrame(texts)

In [12]:
import re

cleanDf = (re.sub("[^A-ZÄÖÜa-zäüöß']+", ' ', str(row)).lower() for row in df[0])

In [13]:
from collections import Counter

word_counter = Counter()

for text in cleanDf:
    word_counter.update(text.split())

In [14]:
word_countDf = pd.DataFrame(word_counter.most_common(), columns = ["word", "count_complete"])

## counter of aufklaerung + merge

In [15]:
# preperation
df = pd.DataFrame(texts_aufklaerung)
cleanDf = (re.sub("[^A-ZÄÖÜa-zäüöß']+", ' ', str(row)).lower() for row in df[0])

# counter
word_counter = Counter()
for text in cleanDf:
    word_counter.update(text.split())

# dataframe    
word_countDf_aufklaerung = pd.DataFrame(word_counter.most_common(), columns = ["word", "count_aufklaerung"])

In [16]:
newDf = word_countDf.merge(word_countDf_aufklaerung, on ="word", how ="left")

## counter of kunstepoche + merge

In [17]:
# preperation
df = pd.DataFrame(texts_kunstepoche)
cleanDf = (re.sub("[^A-ZÄÖÜa-zäüöß']+", ' ', str(row)).lower() for row in df[0])

# counter
word_counter = Counter()
for text in cleanDf:
    word_counter.update(text.split())

# dataframe    
word_countDf_kunstepoche = pd.DataFrame(word_counter.most_common(), columns = ["word", "count_kunstepoche"])

In [18]:
finalDf = newDf.merge(word_countDf_kunstepoche, on ="word", how ="left")

In [19]:
Df = finalDf.fillna(0)

In [20]:
sum_complete = sum(Df["count_complete"])

Df["rel_count_complete"] = Df["count_complete"]/sum_complete
Df.head()

,word,count_complete,count_aufklaerung,count_kunstepoche,rel_count_complete
0,o,36447,16138.0,20309.0,0.006673
1,sehen,31617,16298.0,15319.0,0.005788
2,leben,27259,11569.0,15690.0,0.004990
3,gott,27069,13503.0,13566.0,0.004956
4,schön,24208,10566.0,13642.0,0.004432


In [21]:
sum_aufklaerung = sum(Df["count_aufklaerung"])

Df["rel_count_aufklaerung"] = Df["count_aufklaerung"]/sum_aufklaerung
Df.head()

,word,count_complete,count_aufklaerung,count_kunstepoche,rel_count_complete,rel_count_aufklaerung
0,o,36447,16138.0,20309.0,0.006673,0.006998
1,sehen,31617,16298.0,15319.0,0.005788,0.007067
2,leben,27259,11569.0,15690.0,0.004990,0.005017
3,gott,27069,13503.0,13566.0,0.004956,0.005855
4,schön,24208,10566.0,13642.0,0.004432,0.004582


In [22]:
sum_kunstepoche = sum(Df["count_kunstepoche"])

Df["rel_count_kunstepoche"] = Df["count_kunstepoche"]/sum_kunstepoche
Df.head()

,word,count_complete,count_aufklaerung,count_kunstepoche,rel_count_complete,rel_count_aufklaerung,rel_count_kunstepoche
0,o,36447,16138.0,20309.0,0.006673,0.006998,0.006435
1,sehen,31617,16298.0,15319.0,0.005788,0.007067,0.004854
2,leben,27259,11569.0,15690.0,0.004990,0.005017,0.004971
3,gott,27069,13503.0,13566.0,0.004956,0.005855,0.004298
4,schön,24208,10566.0,13642.0,0.004432,0.004582,0.004322


In [23]:
Df.to_csv("word_freq_lyrik.csv", encoding = "utf8")